In [15]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.models import model_from_json
import os
import numpy
from keras import backend
backend.set_image_data_format('channels_first')

In [2]:
seed = 5
numpy.random.seed(seed)

In [5]:
'''
this part is required to solve
AttributeError: module 'tensorflow_core._api.v2.config' has no attribute 'experimental_list_devices'.
This error occurs because the experimental_list_devices is deprecated in tf 2.1.
'''
import tensorflow as tf
import keras.backend.tensorflow_backend as tfback
print("tf.__version__ is", tf.__version__)
print("tf.keras.__version__ is:", tf.keras.__version__)
def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings).

    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]

tfback._get_available_gpus = _get_available_gpus

tf.__version__ is 2.1.0
tf.keras.__version__ is: 2.2.4-tf


In [6]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train/255
x_test = x_test/255
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [10]:
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=(3,32,32), padding='same', activation='relu'))
model.add(Dropout(.2))
model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(Dropout(.2))
model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(Dropout(.2))
model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dropout(.2))
model.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(.2))
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(.2))
model.add(Dense(num_classes, activation='softmax'))
epochs=25
lr=.01
decay=lr/epochs
sgd = SGD(lr=lr, momentum=.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_5 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 32, 16, 16)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 64, 16, 16)        18496     
_________________________________________________________________
dropout_6 (Dropout)          (None, 64, 16, 16)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 64, 16, 16)       

In [11]:
model.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=epochs, batch_size=32, verbose=2)
scores = model.evaluate(x_test,y_test, verbose=0)

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
 - 417s - loss: 1.8812 - accuracy: 0.3044 - val_loss: 1.5063 - val_accuracy: 0.4470
Epoch 2/25
 - 422s - loss: 1.4496 - accuracy: 0.4712 - val_loss: 1.4209 - val_accuracy: 0.4908
Epoch 3/25
 - 433s - loss: 1.2505 - accuracy: 0.5478 - val_loss: 1.2317 - val_accuracy: 0.5569
Epoch 4/25
 - 419s - loss: 1.1205 - accuracy: 0.6007 - val_loss: 1.0619 - val_accuracy: 0.6222
Epoch 5/25
 - 433s - loss: 1.0060 - accuracy: 0.6433 - val_loss: 0.9398 - val_accuracy: 0.6643
Epoch 6/25
 - 417s - loss: 0.9123 - accuracy: 0.6762 - val_loss: 0.9067 - val_accuracy: 0.6821
Epoch 7/25
 - 423s - loss: 0.8434 - accuracy: 0.7013 - val_loss: 0.8574 - val_accuracy: 0.6977
Epoch 8/25
 - 402s - loss: 0.7819 - accuracy: 0.7242 - val_loss: 0.7948 - val_accuracy: 0.7229
Epoch 9/25
 - 394s - loss: 0.7333 - accuracy: 0.7405 - val_loss: 0.7818 - val_accuracy: 0.7254
Epoch 10/25
 - 407s - loss: 0.6918 - accuracy: 0.7555 - val_loss: 0.7468 - val_accuracy: 0.7409

In [12]:
print('Accuracy: %.2f%%' % (scores[1]*100))

Accuracy: 78.37%


In [16]:
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('model.h5')
print('Saved model to disk')

Saved model to disk


In [21]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights('model.h5')
print("Loaded model from disk")

Loaded model from disk


In [22]:
scores = model.evaluate(x_test,y_test)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

10000/10000 [==============================] - 19s 2ms/step
accuracy: 79.50%
